In [163]:
import transformers
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [164]:
PRE_TRAINED_MODEL_NAME = 'bert-base-german-cased' # 'distilbert-base-german-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [3]:
sample_text = "Das hier ist ein deutscher Beispieltext. Und ein zweiter."
tokens = tokenizer.tokenize(sample_text) # just tokenizes
token_ids = tokenizer.convert_tokens_to_ids(tokens)
ids = tokenizer.encode(sample_text) # already adds special tokens

print(tokens)
print(token_ids)
print(ids)

['Das', 'hier', 'ist', 'ein', 'deutscher', 'Beispiel', '##text', '.', 'Und', 'ein', 'zweiter', '.']
[295, 702, 127, 39, 2433, 2249, 8859, 26914, 1356, 39, 8266, 26914]
[3, 295, 702, 127, 39, 2433, 2249, 8859, 26914, 1356, 39, 8266, 26914, 4]


In [4]:
tokenizer.sep_token, tokenizer.sep_token_id, tokenizer.cls_token, tokenizer.cls_token_id, tokenizer.pad_token, tokenizer.pad_token_id

('[SEP]', 4, '[CLS]', 3, '[PAD]', 0)

In [5]:
ROOT = Path('/Volumes/INWT/Daten_NLP/') # encrypted folder!
DATA = ROOT / '200707_aachener_zeitung_modified.csv' # text is already minimal preprocessed

In [105]:
class INWT_Dataset(Dataset):

    def __init__(self, df, target, text_base, tokenizer, max_len):
        self.df = df
        self.text_base = text_base
        self.target = target
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, item):
        text = str(self.df.loc[item, self.text_base])
        target = np.array(self.df.loc[item, self.target])

        # hier einfach encode() nehmen? brauche ich die attention_mask etc?
        encoding = self.tokenizer.encode_plus(text,
                                              max_length=self.max_len,
                                              truncation=True,
                                              #return_token_type_ids=False,
                                              pad_to_max_length=True,
                                              return_attention_mask=True,
                                              return_tensors='pt',
                                              )

        return {'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'target': torch.tensor(target, dtype=torch.float).unsqueeze(dim=-1) # unsqueezing so shape (batch_size,1)
                }

In [106]:
df = pd.read_csv(DATA)
df = df.fillna('') # replacing Nan with emtpy string
df.head()

,articleId,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,...,titelH3,wordcount,category,city,text_preprocessed,avgTimeOnPage/wordcount,nr_tokens,mean_token_length,nr_tokens_teaser,nr_tokens_titelH1
0,48620281,21,7,12,7,1012,,112.444444,42.857143,33.333333,...,,769,vm,München/Stuttgart,Frische Luft und Bewegung: Diese Kombination r...,0.146222,796,5.359296,29,9
1,48620381,19,6,11,5,1484,,185.500000,42.105263,31.578947,...,,441,vm,Berlin/Frankfurt/Main,"Der Wecker klingelt, aufstehen! Doch gerade im...",0.420635,452,5.938053,33,8
2,48622639,2,2,2,2,0,,0.000000,0.000000,100.000000,...,,390,vm,Berlin,Eltern auf der Suche nach einem guten Babyphon...,0.000000,396,5.848485,30,7
3,48623085,32,9,20,9,974,,81.166667,37.500000,28.125000,...,,345,vm,Berlin,Spülmaschinentabs sollen kleine Alleskönner se...,0.235266,367,5.594005,30,7
4,48623259,24,2,7,2,3797,,223.352941,70.833333,8.333333,...,,182,vm,Berlin,Make-up hat heutzutage einen Zweck: Es soll da...,1.227214,183,5.622951,22,8


In [107]:
#creating train, dev, test
RANDOM_SEED = 123
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_dev, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED, shuffle=True)
df_train.reset_index(drop=True, inplace=True) # so that index starts with 0 again
df_dev.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
print(df_train.shape, df_dev.shape, df_test.shape)

(712, 36) (89, 36) (90, 36)


In [108]:
df_train.head()

,articleId,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,...,titelH3,wordcount,category,city,text_preprocessed,avgTimeOnPage/wordcount,nr_tokens,mean_token_length,nr_tokens_teaser,nr_tokens_titelH1
0,49583837,28,8,14,7,3680,,262.857143,50.0,28.571429,...,,630,vm,Berlin,Viele Deutsche dürften bis zuletzt die Hoffnun...,0.417234,655,6.097710,27,7
1,51204297,4,0,1,0,417,,139.000000,75.0,0.000000,...,,80,vm,Stuttgart,"Geht in einem Auto ein Assistenzsystem kaputt,...",1.737500,77,6.363636,24,6
2,49230331,4,1,1,1,171,,57.000000,75.0,25.000000,...,,127,vm,Berlin,Vor der Unterschrift unter den Bauvertrag brau...,0.448819,124,6.201613,24,6
3,49825661,5,2,2,2,1501,,500.333333,60.0,40.000000,...,,176,vm,Losheim am See,Beim Kauf von Kettenspray können Motorradbesit...,2.842803,178,5.528090,28,8
4,48897105,5,2,2,2,124,,41.333333,60.0,40.000000,...,,149,vm,Krefeld,"Wer häufiger Schmerzen im Gesicht hat, kann Ze...",0.277405,146,6.410959,25,7


In [182]:
def create_DataLoaders(target, text_base, tokenizer, max_len, batch_size):
    # creating DataSets
    ds_train = INWT_Dataset(df=df_train,
                  target = target,
                  text_base = text_base,
                  tokenizer=tokenizer,
                  max_len = max_len)
    ds_dev = INWT_Dataset(df=df_dev,
                  target = target,
                  text_base = text_base,
                  tokenizer=tokenizer,
                  max_len = max_len)
    ds_test = INWT_Dataset(df=df_test,
                  target = target,
                  text_base = text_base,
                  tokenizer=tokenizer,
                  max_len = max_len)
    
    # creating DataLoaders
    dl_train = DataLoader(ds_train, batch_size=batch_size)#, num_workers=4)
    dl_dev = DataLoader(ds_dev, batch_size=batch_size)#, num_workers=4)
    dl_test = DataLoader(ds_test, batch_size=batch_size)#, num_workers=4)
    
    return dl_train, dl_dev, dl_test

In [212]:
dl_train, dl_dev, dl_test = create_DataLoaders(target = 'pageviews',
                                               text_base = 'teaser', # 'titelH1',
                                               tokenizer = tokenizer, 
                                               max_len = 30,            # change depending on used text_base!
                                               batch_size = 32)

In [213]:
# have a look at one batch
data = next(iter(dl_train))
print(data.keys())
input_ids = data['input_ids']
print(input_ids)
print(input_ids.shape)
attention_mask = data['attention_mask']
print(attention_mask)
print(attention_mask.shape)
print(data['target'].shape)

dict_keys(['text', 'input_ids', 'attention_mask', 'target'])
tensor([[    3,   125, 11032, 23293,  1561,  4766, 26918,  1575,    30,  3836,
         21582, 15767,    21,  6471,  4045, 10184,  2026,  6352,    50,  9371,
          1565, 26897,  4877,     7, 26914,  1309,  3984, 14069,   142,     4],
        [    3, 14092,  7857,  1641,   144,   114,   225,  8389,    88,  3503,
         26088,   147,  8959,     7, 26914,   311,  5503,   287,   620,  2734,
           149, 26914, 13510,   213,   104,  8090, 26918,  1475,   213,     4],
        [    3,  1309,    39,  2843, 10630,  1279, 26918,   259,  1146,    39,
          1020,    81,  9695, 26918,  1547, 24174,   167, 26914,   311, 26500,
          4347,   718,    30,   788, 14069,  5976, 26914,  2019,   788,     4],
        [    3,   125, 20072,   443, 13809, 26902,  1080,  7746,    42,  7746,
         26935,    42,  1616,  6633, 16266,  1281,    80, 26914,  5372,   127,
         16449,  5032,  2945, 26914,   152,  5569, 18192,  1760, 21

## Model

In [141]:
# das ist umständlich (und auch falsch), ich habe stattdessen bereits BertForSequenceClassification genommen
# https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification

#class Bert_regression(nn.Module):
#    
#    def __init__(self, n_outputs): # maybe train pageviews and timeOnPage simultaneously?
#        super(Bert_regression, self).__init__()
#        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
#        self.drop = nn.Dropout(p=0.3)
#        self.out = nn.Linear(self.bert.config.hidden_size, n_outputs)
#
#    def forward(self, input_ids, attention_mask):
#        _, pooled_output = self.bert(input_ids=input_ids,           # das hier ist glaube ich nicht sinnvoll bei mir
#                                     attention_mask=attention_mask)
#        output = self.drop(pooled_output)
#   
#        return self.out(output)

In [ ]:
#model = Bert_regression(n_outputs = 1)

In [ ]:
#model = model.to(device)

In [214]:
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME,
                                                      num_labels = 1, # turns "classification" into regression?
                                                      output_attentions = False,
                                                      output_hidden_states = False,
                                                      )

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

In [ ]:
#model = model.to(device)

In [215]:
# try ut with one batch (model is not trained yet so nothing exciting to be expected)
model(input_ids, attention_mask) # semms good!

(tensor([[ 0.0530],
         [ 0.1250],
         [ 0.0397],
         [ 0.1117],
         [ 0.2243],
         [ 0.0653],
         [ 0.1175],
         [ 0.2201],
         [ 0.1270],
         [-0.0224],
         [-0.1329],
         [ 0.0188],
         [ 0.0862],
         [ 0.1502],
         [ 0.0040],
         [ 0.0829],
         [ 0.0332],
         [ 0.0683],
         [-0.0279],
         [-0.2214],
         [ 0.0340],
         [ 0.1542],
         [ 0.0202],
         [-0.0462],
         [-0.0607],
         [-0.0083],
         [-0.0254],
         [ 0.0145],
         [ 0.0912],
         [ 0.2070],
         [ 0.2703],
         [ 0.0668]], grad_fn=<AddmmBackward>),)

## Training

In [216]:
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.MSELoss()  # mean squared loss

In [221]:
EPOCHS = 25


for epoch in range(EPOCHS):
    print("Epoch", epoch)
    
    ### TRAINING on train
    print("training")
    model = model.train()
    train_losses = []
    
    for nr, d in enumerate(dl_train):
        print("-Batch", nr, end='\r')
        input_ids = d["input_ids"]
        attention_mask = d["attention_mask"]
        targets = d["target"]
        #print(targets.shape)    
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)[0] # stimmt das so? ist [0] die logits?
        #print(outputs.shape)
        
        loss = loss_fn(outputs, targets)
        train_losses.append(loss.item())
        loss.backward()

        #nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()
    
        #print(np.mean(train_losses))
    print("Mean train loss:", np.mean(train_losses))
    
    ### EVALUATING on dev
    print("evaluating")
    model = model.eval()
    eval_losses = []
    
    with torch.no_grad():
        for nr, d in enumerate(dl_dev):
            print("-Batch", nr, end='\r')
            input_ids = d["input_ids"]
            attention_mask = d["attention_mask"]
            targets = d["target"]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)[0] # stimmt das so?
        
            loss = loss_fn(outputs, targets)
            eval_losses.append(loss.item())
            #print(np.mean(eval_losses))
        print("Mean eval loss:", np.mean(eval_losses))

Epoch 0
training
Mean train loss: 31169.826637268066
evaluating
Mean eval loss: 97320.0082397461
Epoch 1
training
Mean train loss: 31157.769926651665
evaluating
Mean eval loss: 97300.71371459961
Epoch 2
training
Mean train loss: 31151.131855840267
evaluating
Mean eval loss: 97283.52858479817
Epoch 3
training
Mean train loss: 31142.364156971806
evaluating
Mean eval loss: 97267.3794555664
Epoch 4
training
Mean train loss: 31131.699822135593
evaluating
Mean eval loss: 97252.37390136719
Epoch 5
training
Mean train loss: 31122.820015948753
evaluating
Mean eval loss: 97237.1313273112
Epoch 6
training
Mean train loss: 31115.45326664137
evaluating
Mean eval loss: 97224.20383707683
Epoch 7
training
Mean train loss: 31111.353743511696
evaluating
Mean eval loss: 97210.33953857422
Epoch 8
training
Mean train loss: 31098.77880859375
evaluating
Mean eval loss: 97197.90098063152
Epoch 9
training
Mean train loss: 31093.607056327488
evaluating
Mean eval loss: 97185.94038899739
Epoch 10
training
Mean tr

## predict dev set

In [218]:
import scipy.stats as st

In [219]:
_, dl_dev_oneBatch, _ = create_DataLoaders(target = 'pageviews',
                                               text_base = 'titelH1', # 'teaser'
                                               tokenizer = tokenizer, 
                                               max_len = 20,            # change depending on used text_base!
                                               batch_size = 89) # for now: just one batch

In [220]:
with torch.no_grad():
    for d in dl_dev_oneBatch:
        pred_dev = model(input_ids=d["input_ids"], attention_mask=d["attention_mask"])[0] # just the logits?
        y_dev = d["target"]
        
        pred_dev = pred_dev.squeeze()
        y_dev = y_dev.squeeze()
        print(pred_dev)
        print(y_dev)
        print(st.pearsonr(pred_dev, y_dev))

tensor([12.2933, 12.3086, 12.3034, 12.2823, 12.2906, 12.2941, 12.3136, 12.2987,
        12.2861, 12.2873, 12.2507, 12.2997, 12.2819, 12.2853, 12.3088, 12.2795,
        12.2993, 12.2814, 12.2881, 12.3007, 12.2934, 12.2879, 12.2927, 12.2814,
        12.2832, 12.3043, 12.3022, 12.2989, 12.2966, 12.3052, 12.2808, 12.3051,
        12.2821, 12.2927, 12.2764, 12.2815, 12.2991, 12.2824, 12.2783, 12.2737,
        12.2980, 12.3045, 12.2785, 12.2990, 12.2982, 12.3028, 12.2725, 12.2922,
        12.2856, 12.2970, 12.2983, 12.2904, 12.2839, 12.2974, 12.2993, 12.2653,
        12.3095, 12.2745, 12.2972, 12.2916, 12.3087, 12.2794, 12.2994, 12.2933,
        12.2878, 12.3078, 12.2545, 12.2969, 12.2767, 12.2955, 12.3029, 12.3008,
        12.2982, 12.2705, 12.2950, 12.2845, 12.2903, 12.3097, 12.3059, 12.2845,
        12.2968, 12.2984, 12.3076, 12.3052, 12.2739, 12.2729, 12.2879, 12.3032,
        12.2816])
tensor([3.0470e+03, 3.3000e+01, 2.0000e+01, 6.4000e+01, 1.0000e+00, 1.2000e+01,
        2.0000e+00, 2.

## Kommentare:
* Das Modell lernt nicht! Liegt das einfach an der geringen Datenmenge oder ist im Training was falsch?
* anderer Optimizer? Ist Loss richtig berechnet?
* Oder liegt das Problem bereits beim DataSet, Dataloader?